In [1]:
import re
import polars as pl

In [2]:
# Function to parse the log file
def parse_log_file(log_file_path):
    user_list = []
    token_list = []
    
    with open(log_file_path, "r") as f:
        for line in f:
            # Check if the line contains the word 'confirm'
            if 'confirm' in line:
                # Extract user and token using regular expressions
                user_match = re.search(r'trader (\d+): confirm', line)
                token_match = re.search(r'token (\w+)\.', line)
                
                if user_match and token_match:
                    user = user_match.group(1)
                    token = token_match.group(1)
                    
                    user_list.append(user)
                    token_list.append(token)

    # Create a Polars DataFrame
    df = pl.DataFrame(
        {
            "user": user_list,
            "token": token_list
        }
    )

    return df


In [3]:
# Path to your log file
log_file_path = "log_hft.txt"

# Parse the log file and create a Polars DataFrame
df = parse_log_file(log_file_path)

# Show the DataFrame
print(df)


shape: (724, 2)
┌──────┬────────────────┐
│ user ┆ token          │
│ ---  ┆ ---            │
│ str  ┆ str            │
╞══════╪════════════════╡
│ 1    ┆ INVES000100001 │
│ 1    ┆ INVEB000100002 │
│ 1    ┆ INVES000100003 │
│ 1    ┆ INVES000100003 │
│ …    ┆ …              │
│ 1982 ┆ BBBBB198200246 │
│ 1981 ┆ AAAAS198100482 │
│ 1982 ┆ BBBBB198200248 │
│ 1    ┆ INVES000100230 │
└──────┴────────────────┘


In [7]:
# Count unique users per token
users_by_token = df.groupby("token").agg(
    pl.col("user").n_unique().alias("unique_users")
)

In [8]:
users_by_token.filter(pl.col("unique_users") > 1)

token,unique_users
str,u32


In [5]:
# Count unique tokens per user
tokens_by_user = df.groupby("user").agg(
    pl.col("token").n_unique().alias("unique_tokens")
)

print(tokens_by_user)

shape: (3, 2)
┌──────┬───────────────┐
│ user ┆ unique_tokens │
│ ---  ┆ ---           │
│ str  ┆ u32           │
╞══════╪═══════════════╡
│ 1    ┆ 230           │
│ 1982 ┆ 96            │
│ 1981 ┆ 125           │
└──────┴───────────────┘


In [11]:
def read_token_crossing_log(file_path):
    buy_tokens = []
    sell_tokens = []
    crossing_prices = []
    volumes = []
    
    # Regex pattern to extract buy token, sell token, crossing price, and volume
    pattern = r"Orders \(b'(.*?)', b'(.*?)'\) crossed at price (\d+), volume (\d+)"
    
    # Read log file line-by-line and extract relevant information
    with open(file_path, 'r') as f:
        for line in f:
            if 'root.process_cross' in line:
                match = re.search(pattern, line)
                if match:
                    buy_token, sell_token, crossing_price, volume = match.groups()
                    buy_tokens.append(buy_token)
                    sell_tokens.append(sell_token)
                    crossing_prices.append(int(crossing_price))
                    volumes.append(int(volume))
                    
    # Create a DataFrame using the extracted information
    df_crossing = pl.DataFrame({
        'buy_token': buy_tokens,
        'sell_token': sell_tokens,
        'crossing_price': crossing_prices,
        'volume': volumes
    })
    
    return df_crossing

In [12]:
# Example usage
file_path = 'log_exchange_server.txt'  # Replace with the actual path to your log file
df_crossing = read_token_crossing_log(file_path)
print(df_crossing)

shape: (124, 4)
┌────────────────┬────────────────┬────────────────┬────────┐
│ buy_token      ┆ sell_token     ┆ crossing_price ┆ volume │
│ ---            ┆ ---            ┆ ---            ┆ ---    │
│ str            ┆ str            ┆ i64            ┆ i64    │
╞════════════════╪════════════════╪════════════════╪════════╡
│ INVES000100003 ┆ INVEB000100002 ┆ 1700000        ┆ 1      │
│ AAAAS198100001 ┆ INVEB000100005 ┆ 1710000        ┆ 1      │
│ BBBBS198200001 ┆ INVEB000100004 ┆ 1690000        ┆ 1      │
│ AAAAB198100007 ┆ BBBBS198200004 ┆ 1680000        ┆ 1      │
│ …              ┆ …              ┆ …              ┆ …      │
│ INVEB000100225 ┆ INVES000100217 ┆ 1270000        ┆ 1      │
│ INVEB000100227 ┆ INVES000100218 ┆ 1270000        ┆ 1      │
│ AAAAS198100473 ┆ INVEB000100226 ┆ 1250000        ┆ 1      │
│ AAAAS198100478 ┆ BBBBB198200246 ┆ 1270000        ┆ 1      │
└────────────────┴────────────────┴────────────────┴────────┘


In [18]:
# Joining to get buyer information
final_df = (
    df_crossing
    .join(df, left_on="buy_token", right_on="token", how="left")
    .rename({"user": "buyer"})
    .join(df, left_on="sell_token", right_on="token", how="left")
    .rename({"user": "seller"})
    )


In [21]:
print(final_df)


shape: (338, 6)
┌────────────────┬────────────────┬────────────────┬────────┬───────┬────────┐
│ buy_token      ┆ sell_token     ┆ crossing_price ┆ volume ┆ buyer ┆ seller │
│ ---            ┆ ---            ┆ ---            ┆ ---    ┆ ---   ┆ ---    │
│ str            ┆ str            ┆ i64            ┆ i64    ┆ str   ┆ str    │
╞════════════════╪════════════════╪════════════════╪════════╪═══════╪════════╡
│ INVES000100003 ┆ INVEB000100002 ┆ 1700000        ┆ 1      ┆ 1     ┆ 1      │
│ INVES000100003 ┆ INVEB000100002 ┆ 1700000        ┆ 1      ┆ 1     ┆ 1      │
│ INVES000100003 ┆ INVEB000100002 ┆ 1700000        ┆ 1      ┆ 1     ┆ 1      │
│ INVES000100003 ┆ INVEB000100002 ┆ 1700000        ┆ 1      ┆ 1     ┆ 1      │
│ …              ┆ …              ┆ …              ┆ …      ┆ …     ┆ …      │
│ AAAAS198100473 ┆ INVEB000100226 ┆ 1250000        ┆ 1      ┆ 1981  ┆ 1      │
│ AAAAS198100473 ┆ INVEB000100226 ┆ 1250000        ┆ 1      ┆ 1981  ┆ 1      │
│ AAAAS198100473 ┆ INVEB000100226 ┆ 

In [32]:
self_crossing_df = final_df.filter(
    (pl.col("buyer") == pl.col("seller")) &
    (pl.col("buyer") != "1")
)


In [33]:
self_crossing_df.write_csv("self_crossing.csv")